In [1]:
import codecs
import tensorflow as tf
import numpy as np

print(tf.__version__)

import matplotlib.pyplot as plt
%matplotlib inline

2.12.0


# Загрузка датасета

In [2]:
data_fpath = tf.keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = codecs.open(data_fpath, 'r', encoding='utf8').read()
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



# Конвертация символов в индексы

In [3]:
vocab = sorted(set(text))
VOCAB_SIZE = len(vocab) # Количество слов в словаре




print(f'Vocab: {vocab}')
print(f'{VOCAB_SIZE} unique characters')

char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

print(f'Example of the encoded text: {text_as_int[:13]}')

Vocab: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65 unique characters
Example of the encoded text: [18 47 56 57 58  1 15 47 58 47 64 43 52]


# Подготовка датасета

In [4]:
SEQ_LEN = 1000
BATCH_SIZE = 64

input_seqs = []
target_seqs = []

num_seqs = len(text_as_int) // (SEQ_LEN + 1)
for i in range(num_seqs):
    seq = text_as_int[i:i+SEQ_LEN+1]
    input_seqs.append(np.array(seq[:-1]))
    target_seqs.append(np.array(seq[1:]))
    
input_seqs = np.array(input_seqs)
target_seqs = np.array(target_seqs)

input_seqs = input_seqs[:(len(input_seqs)//BATCH_SIZE)*BATCH_SIZE]
target_seqs = target_seqs[:(len(input_seqs)//BATCH_SIZE)*BATCH_SIZE]

print(f'Input: {[idx2char[i] for i in input_seqs[0][:15]]}')
print(f'Target: {[idx2char[i] for i in target_seqs[0][:15]]}')

Input: ['F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n']
Target: ['i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n', 'B']


# Функция построения модели

In [19]:
def build_model(batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(VOCAB_SIZE, 256, batch_input_shape=(batch_size, None)),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.GRU(256, return_sequences=True, stateful=True),
        tf.keras.layers.Dense(VOCAB_SIZE),
    ])
    model.build()
    return model

model = build_model(BATCH_SIZE)

# Обучение модели

In [ ]:
EPOCHS = 100

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

history = model.fit(input_seqs,
                    target_seqs,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE)

Epoch 1/100
17/17 [==============================] - 7s 343ms/step - loss: 3.7050
Epoch 2/100
17/17 [==============================] - 6s 342ms/step - loss: 3.0363
Epoch 3/100
17/17 [==============================] - 6s 344ms/step - loss: 2.7804
Epoch 4/100
17/17 [==============================] - 6s 341ms/step - loss: 2.4702
Epoch 5/100
17/17 [==============================] - 6s 345ms/step - loss: 2.2498
Epoch 6/100
17/17 [==============================] - 6s 344ms/step - loss: 2.0879
Epoch 7/100
17/17 [==============================] - 6s 344ms/step - loss: 1.9372
Epoch 8/100
17/17 [==============================] - 6s 345ms/step - loss: 1.7813
Epoch 9/100
17/17 [==============================] - 6s 345ms/step - loss: 1.6108
Epoch 10/100
17/17 [==============================] - 6s 345ms/step - loss: 1.4170
Epoch 11/100
17/17 [==============================] - 6s 345ms/step - loss: 1.1985
Epoch 12/100
17/17 [==============================] - 6s 344ms/step - loss: 0.9608
Epoch 13/100


# Создание модели для инференса

In [7]:
model_inf = build_model(1)

for i in range(len(model_inf.layers)):
    for j in range(len(model_inf.layers[i].weights)):
        model_inf.layers[i].weights[j].assign(model.layers[i].weights[j])

# Функция генерации текста

In [15]:
def generate_text(model, seed, out_len):
    
    text_generated = []
    
    # Обнуляем состояние модели
    model.reset_states()
    
    # Конвертируем входную цепочку в индексы
    inp = np.array([char2idx[s] for s in seed])
    
    for i in range(out_len):
        
        # Получаем предсказания для входной цепочки inp
        # pred - матрица размерности (длина цепочки, распределение по классам)
        # На первой итерации цикла длина цепочки равна длине seed, а затем длина равна 1
        pred = model(inp[None, ...])[0]
        
        # Для получения символа сэмплируем из распределения
        # Больная температура соответствуем более случайному предсказанию символа
        temperature = 1.0
        pred = pred / temperature
        pred_c = tf.random.categorical(pred, num_samples=1)[-1][0].numpy()
        
        text_generated.append(idx2char[pred_c])
        
        # Новый вход -- только что сгенерированный символ
        inp = np.array([pred_c])
    
    return (seed + ''.join(text_generated))

In [16]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc']
             )

# Запуск генератора текста
Запускаем генерацию текста, передавая на вход желаемое начало цепочки seed.

In [17]:
print(generate_text(model_inf, seed=u'MONTAGUE:', out_len=500))

MONTAGUE:
Against him first: he's a very dog to the commonalty.

Second Citizen:
Consider you what services he has done for his country?

First Citizen:
Very well; and could be content to give him good
report fort, but that he pays himself with being proud.

Second Citizen:
Nay, but speak not maliciously, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superflui
